In [ ]:
pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 56.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import fitz



---

##2014-2015 (add in state)


In [ ]:
def initialize_data():
    return {
        "name": [],
        "tot": [],
        "domain": [],
        "subgroup": [],
        "% demonstrating readiness": [],
        "% developing readiness": [],
        "% emerging readiness": []
    }

In [ ]:
def extract_county_name(text):
    county_pattern = r'([A-Za-z\s]+)’s kindergarteners'
    match_county = re.search(county_pattern, text)
    if match_county:
        return match_county.group(1).strip()
    return ""

In [ ]:
def extract_total_students(text):
    students_pattern = r'Kindergarten Students\s+(\d+)'
    match_students = re.search(students_pattern, text)
    if match_students:
        return int(match_students.group(1))
    return 0

In [ ]:
def extract_domain_data(text, data, name, tot):
  domains = ["Language & Literacy", "Mathematics", "Physical Well-being & Motor Development ", "Social Foundations"]
  for domain in domains:
    domain_pattern = re.escape(domain).replace(r'\ ', r'\s*') + r'\s*\((\d+)%\)'
    match_domain = re.search(domain_pattern, text)
    if match_domain:
        percentage = int(match_domain.group(1))
        data["domain"].append(domain)
        data["subgroup"].append("all")
        data["% demonstrating readiness"].append(percentage)
        data["% developing readiness"].append("NA")
        data["% emerging readiness"].append("NA")
        data["name"].append(name)
        data["tot"].append(tot)
  return data

In [ ]:
def extract_overall_readiness(text, data, name, tot):
    demo_pattern = r'(\d+)%\s*demonstrating'
    match_demo = re.search(demo_pattern, text)
    if match_demo:
        demo_readiness = int(match_demo.group(1))
        data["domain"].append("overall")
        data["subgroup"].append("all")
        data["% demonstrating readiness"].append(demo_readiness)
        # data["% developing readiness"].append('NA')
        # data["% emerging readiness"].append('NA')
        data["name"].append(name)
        data["tot"].append(tot)

    readiness_pattern = re.compile(r'(\d+)%\s+(\d+)%\s+developing readiness\s+emerging readiness', re.DOTALL)
    match_readiness = readiness_pattern.search(text)
    if match_readiness:
        developing_readiness = int(match_readiness.group(1))
        emerging_readiness = int(match_readiness.group(2))
        data["% developing readiness"].append(developing_readiness)
        data["% emerging readiness"].append(emerging_readiness)

In [ ]:
def extract_subgroup_readiness(text, data, name, tot):
    subgroups_before_name = [
        "Children w/ Disability", "Children w/o Disability",
        "English Language Learners", "English Proficient",
        "Children from Low-Income Households", "Children from Mid-/High Income Households"
    ]

    subgroups_after_name = [
        "American Indian", "Asian", "African American",
        "Native Hawaiian/Pacific Islander", "White", "Hispanic", "Two or More Races"
    ]

    # Regex pattern for percentage
    percentage_pattern = r'(\d+|\*)%'

    # Iterate over all subgroups
    all_subgroups = subgroups_before_name + subgroups_after_name
    for subgroup in all_subgroups:
        if subgroup in subgroups_after_name:
            subgroup_pattern = r'\b{}\b(?: \((.*?)\)|\s+({}))'.format(re.escape(subgroup), percentage_pattern)
            match = re.search(subgroup_pattern, text)
            if match:
                demo_readiness = match.group(1) if match.group(1) else '*'
        elif subgroup in subgroups_before_name:
            subgroup_pattern = r'{}(?:\s+{})'.format(percentage_pattern, re.escape(subgroup))
            match = re.search(subgroup_pattern, text)
            demo_readiness = match.group(1) if match else '*'

        data["name"].append(name)
        data["tot"].append(tot)
        data["domain"].append("overall")
        data["subgroup"].append(subgroup)
        data["% demonstrating readiness"].append(demo_readiness)
        data["% developing readiness"].append('NA')
        data["% emerging readiness"].append('NA')

In [ ]:
def extract_data(pdf_path, page_num):
  data = initialize_data()
  pdf_document = fitz.open(pdf_path)

  try:
    page = pdf_document.load_page(page_num)
    text = page.get_text("text")

    name = extract_county_name(text)
    tot = extract_total_students(text)

    extract_domain_data(text, data, name, tot)

    extract_overall_readiness(text, data, name, tot)

    extract_subgroup_readiness(text, data, name, tot)
  finally:
    pdf_document.close()
  df = pd.DataFrame(data)
  return df


In [ ]:
pdf_path = "/content/readinessmatters2014-2015.pdf"
page_num = 11  # For page 12

# Extract data from PDF and store in DataFrame
extract_data(pdf_path, page_num)


,name,tot,domain,subgroup,% demonstrating readiness,% developing readiness,% emerging readiness
0,Allegany County,645,Language & Literacy,all,44,NA,NA
1,Allegany County,645,Mathematics,all,43,NA,NA
2,Allegany County,645,Physical Well-being & Motor Development,all,53,NA,NA
3,Allegany County,645,Social Foundations,all,54,NA,NA
4,Allegany County,645,overall,all,47,34,19
5,Allegany County,645,overall,Children w/ Disability,20,NA,NA
6,Allegany County,645,overall,Children w/o Disability,51,NA,NA
7,Allegany County,645,overall,English Language Learners,*,NA,NA
8,Allegany County,645,overall,English Proficient,47,NA,NA
9,Allegany County,645,overall,Children from Low-Income Households,37,NA,NA


In [ ]:
def extract_pages(pdf_path, start, end):
  data = initialize_data()
  pdf_document = fitz.open(pdf_path)

  try:
    for page_num in range(start, end + 1):
      page = pdf_document.load_page(page_num)
      text = page.get_text("text")

      name = extract_county_name(text)
      tot = extract_total_students(text)

      extract_domain_data(text, data, name, tot)

      extract_overall_readiness(text, data, name, tot)

      extract_subgroup_readiness(text, data, name, tot)
  finally:
    pdf_document.close()
  df = pd.DataFrame(data)
  return df

In [ ]:
pdf_path = "/content/readinessmatters2014-2015.pdf"
start = 11
end = 34

df_1415 = extract_pages(pdf_path, start, end)
df_1415

,name,tot,domain,subgroup,% demonstrating readiness,% developing readiness,% emerging readiness
0,Allegany County,645,Language & Literacy,all,44,NA,NA
1,Allegany County,645,Mathematics,all,43,NA,NA
2,Allegany County,645,Physical Well-being & Motor Development,all,53,NA,NA
3,Allegany County,645,Social Foundations,all,54,NA,NA
4,Allegany County,645,overall,all,47,34,19
...,...,...,...,...,...,...,...
427,Worcester County,482,overall,African American,24%,NA,NA
428,Worcester County,482,overall,Native Hawaiian/Pacific Islander,*,NA,NA
429,Worcester County,482,overall,White,57%,NA,NA
430,Worcester County,482,overall,Hispanic,32%,NA,NA


In [ ]:
df_1415.to_csv("MD_1415.csv")

have not added state data



---

## 2015-2016

1. Get the totals (p1) for 1 "name" into a df.
2. Get the numbers (p2) for 1 "name" into a df.
3. Get the pers (p3) for 1 "name into a df.
4. Merge them together.
5. Repeat 1-4 with pages 6, 7, 8
6. Repeat with pages 9, 10, 11

2, 7, 12 -> pages w/ numbers are page 2 + 5

1, 6, 11 -> pages w/ tots are page 1 + 5

3, 8, 13 -> pages w/ pers are page 3 + 5

Totals

In [ ]:
def clean_tots(tots_text):
  tots_lines = tots_text.split("\n")
  remove = ["Frequency", "Percent"]
  tots_lines = [word for word in tots_lines if word not in remove]

  counters = {"No": 0, "Yes": 0}
  replacement_texts = {
      "No": ["Free and Reduced Price Meals - No", "Special Education - No", "English Language Learners - No"],
      "Yes": ["Free and Reduced Price Meals - Yes", "Special Education - Yes", "English Language Learners - Yes"]
  }

  clean_tots = []

  for line in tots_lines:
    if line in counters:
      counters[line] += 1
      if counters[line] <= len(replacement_texts[line]):
        clean_tots.append(replacement_texts[line][counters[line] - 1])
      else:
        clean_tots.append(line)
    else:
        clean_tots.append(line)
  return clean_tots

In [ ]:
def build_dftots(clean_tots):
  df_1516_tots = pd.DataFrame(columns = ["name", "subgroup", "n_tested", "pers_tested"])
  name = ""
  for line in clean_tots:
    if " Data File Summary" in line:
      name = line.split("Data")[0].strip()
      break

  clean_tots = [line.replace(',', '') for line in clean_tots]

  row_index = 0
  i = 0

  while i < len(clean_tots):
    if not clean_tots[i].isdigit():
      if i + 1 < len(clean_tots) and not clean_tots[i + 1].isdigit():
        subgroup = f"{clean_tots[i]} - {clean_tots[i + 1]}"
        i += 2
      else:
        subgroup = clean_tots[i]
        i += 1
      while i < len(clean_tots) and clean_tots[i].isdigit():
        df_1516_tots.loc[row_index] = {
            "name": name,
            "subgroup": subgroup,
            "n_tested": clean_tots[i],
            "pers_tested": clean_tots[i + 1]
        }
        i += 2
      row_index += 1

  df_1516_tots.replace("American Indian/Alaska Native", "American Indian/Alaskan Native", inplace=True)
  df_1516_tots.replace("Black/African American", "African American", inplace=True)
  df_1516_tots.replace("Native Hawaiian/Other Pacific Islander", "Native Hawaiian/Pacific Islander", inplace=True)
  df_1516_tots.replace("Hispanic/Latino", "Hispanic", inplace=True)
  df_1516_tots.replace("Males", "Male", inplace=True)
  df_1516_tots.replace("Females", "Female", inplace=True)
  df_1516_tots.replace("Prekindergarten", "Pre-Kindergarten", inplace=True)
  df_1516_tots.replace("Home/Informal Care", "Home / Informal Care", inplace=True)
  df_1516_tots.replace("Non-Public Nursery School", "Non-public Nursery", inplace=True)
  # df_1516_tots.replace("", "", inplace=True)
  # df_1516_tots.replace("", "", inplace=True)


  return df_1516_tots

Nums

In [ ]:
def clean_nums(nums_text):
  nums_lines = nums_text.split("\n")

  counters = {"No": 0, "Yes": 0}
  replacement_texts = {
      "No": ["Special Education - No", "English Language Learners - No", "Free and Reduced Price Meals - No"],
      "Yes": ["Special Education - Yes", "English Language Learners - Yes", "Free and Reduced Price Meals - Yes"]
  }

  clean_lines = []

  for line in nums_lines:
    if line in counters:
      counters[line] += 1
      if counters[line] <= len(replacement_texts[line]):
        clean_lines.append(replacement_texts[line][counters[line] - 1])
      else:
        clean_lines.append(line)
    else:
        clean_lines.append(line)
  return clean_lines

In [ ]:
def build_dfnums(clean_lines):
  df_1516 = pd.DataFrame(columns=["name", "subgroup", "domain", "n_demo", "n_notdemo", "n_appr", "n_emer"])

  name = ""
  for line in clean_lines:
    # if " - Number of Kindergarten Students" in line:
    #   name = line.split(" - ")[0].strip()
    # if re.search(r" -?\s?Number of Kindergarten Students", line):
    #   name = re.split(r" -?\s?Number of Kindergarten Students", line)[0].strip()
    #   break
    # if "Number of Kindergarten Students" in line.replace(" -", "").replace("- ", "").replace("-", "").strip():
    #   name = line.split(" - Number of Kindergarten Students")[0].strip()
    #   break
    if " - Number of Kindergarten Students" in line:
      name = line.split(" - ")[0].strip()
      break
    elif "- Number of Kindergarten Students" in line:
      name = line.split("- ")[0].strip()
      break
    elif " -Number of Kindergarten Students" in line:
      name = line.split(" -")[0].strip()
      break

  domains = ["lanlit", "math", "sel", "phy", "overall"]
  row_index = 0
  domain_index = 0
  i = 0

  clean_lines = [line.replace(',', '') for line in clean_lines]

  while i < len(clean_lines):
      # Check for non-numeric entry (subgroup name)
      if not clean_lines[i].replace('.', '', 1).isdigit() and clean_lines[i] != '*':
          subgroup = clean_lines[i]
          i += 1
          # Process numeric entries
          while i < len(clean_lines) and (clean_lines[i].replace('.', '', 1).isdigit() or clean_lines[i] == '*'):
              domain = domains[domain_index]
              if domain == "overall":
                  # Handle 'overall' domain with 3 values
                  df_1516.loc[row_index] = {
                      "name": name,
                      "subgroup": subgroup,
                      "domain": domain,
                      "n_demo": clean_lines[i],
                      "n_appr": clean_lines[i+1],
                      "n_emer": clean_lines[i+2]
                  }
                  i += 3
              else:
                  # Handle other domains with 2 values
                  df_1516.loc[row_index] = {
                      "name": name,
                      "subgroup": subgroup,
                      "domain": domain,
                      "n_demo": clean_lines[i],
                      "n_notdemo": clean_lines[i+1]
                  }
                  i += 2

              row_index += 1
              domain_index += 1

              # Check if we've exhausted the domains
              if domain_index >= len(domains):
                  break

          # Reset domain index for the next subgroup
          domain_index = 0
      else:
          i += 1

  # Reset index of DataFrame if needed
  df_1516.reset_index(drop=True, inplace=True)

  return df_1516

Pers

In [ ]:
def clean_pers(pers_text):
  pers_list = pers_text.split("\n")

  counters = {"No": 0, "Yes": 0}
  replacement_texts = {
      "No": ["Special Education - No", "English Language Learners - No", "Free and Reduced Price Meals - No"],
      "Yes": ["Special Education - Yes", "English Language Learners - Yes", "Free and Reduced Price Meals - Yes"]
  }

  clean_pers = []

  for line in pers_list:
    if line in counters:
      counters[line] += 1
      if counters[line] <= len(replacement_texts[line]):
        clean_pers.append(replacement_texts[line][counters[line] - 1])
      else:
        clean_pers.append(line)
    else:
        clean_pers.append(line)
  return clean_pers


In [ ]:
def build_persdf(clean_pers):
  df_1516_pers = pd.DataFrame(columns=["name", "subgroup", "domain", "pers_demo", "pers_notdemo", "pers_appr", "pers_emer"])

  name = ""
  for line in clean_pers:
    # if " - Percentage of Kindergarten Students" in line:
    #   name = line.split(" - ")[0].strip()
    # if re.search(r" -?\s?Percentage of Kindergarten Students", line):
    #   name = re.split(r" -?\s?Percentage of Kindergarten Students", line)[0].strip()
    #   break
    # if "Percentage of Kindergarten Students" in line.replace(" -", "").replace("- ", "").replace("-", "").strip():
    #   name = line.split(" - Percentage of Kindergarten Students")[0].strip()
    #   break
    if " - Percentage of Kindergarten Students" in line:
      name = line.split(" - ")[0].strip()
      break
    elif "- Percentage of Kindergarten Students" in line:
      name = line.split("- ")[0].strip()
      break
    elif " -Percentage of Kindergarten Students" in line:
      name = line.split(" -")[0].strip()
      break


  domains = ["lanlit", "math", "sel", "phy", "overall"]
  row_index = 0
  domain_index = 0
  i = 0

  clean_pers = [line.replace(',', '') for line in clean_pers]

  while i < len(clean_pers):
      # Check for non-numeric entry (subgroup name)
      if not clean_pers[i].replace('.', '', 1).isdigit() and clean_pers[i] != '*':
          subgroup = clean_pers[i]
          i += 1

          # Process numeric entries
          while i < len(clean_pers) and (clean_pers[i].replace('.', '', 1).isdigit() or clean_pers[i] == '*'):
              domain = domains[domain_index]
              if domain == "overall":
                  # Handle 'overall' domain with 3 values
                  df_1516_pers.loc[row_index] = {
                      "name": name,
                      "subgroup": subgroup,
                      "domain": domain,
                      "pers_demo": clean_pers[i],
                      "pers_appr": clean_pers[i+1],
                      "pers_emer": clean_pers[i+2]
                  }
                  i += 3
              else:
                  # Handle other domains with 2 values
                  df_1516_pers.loc[row_index] = {
                      "name": name,
                      "subgroup": subgroup,
                      "domain": domain,
                      "pers_demo": clean_pers[i],
                      "pers_notdemo": clean_pers[i+1]
                  }
                  i += 2

              row_index += 1
              domain_index += 1

              # Check if we've exhausted the domains
              if domain_index >= len(domains):
                  break

          # Reset domain index for the next subgroup
          domain_index = 0
      else:
          i += 1

  # Reset index of DataFrame if needed
  df_1516_pers.reset_index(drop=True, inplace=True)

  return df_1516_pers

Merge

In [ ]:
def merge(df_1516, df_1516_pers, df_1516_tots):
  merged_df = pd.merge(df_1516, df_1516_pers,
                     on = ["name", "subgroup", "domain"])
  merged_final = pd.merge(merged_df, df_1516_tots,
                          on = ["name", "subgroup"], how = "outer")
  return merged_final

  #return merged_df

Loop

In [ ]:
def get_tables(pdf_path):
    pdf_doc = fitz.open(pdf_path)
    try:
        df_1516_tots_list = []
        df_1516_nums_list = []
        df_1516_pers_list = []

        for i in range(0, pdf_doc.page_count, 5):
            # tots are on pages 1, 6, 11, ...
            if i < pdf_doc.page_count:
                tot_page = pdf_doc.load_page(i)
                tots_text = tot_page.get_text("text")
                clean_tots_data = clean_tots(tots_text)
                df_1516_tots = build_dftots(clean_tots_data)
                df_1516_tots_list.append(df_1516_tots)
        #return clean_tots_data

            # nums are on pages 2, 7, 12, ...
            if (i + 1) < pdf_doc.page_count:
                nums_page = pdf_doc.load_page(i + 1)
                nums_text = nums_page.get_text("text")
                clean_nums_data = clean_nums(nums_text)
                df_1516_nums = build_dfnums(clean_nums_data)
                df_1516_nums_list.append(df_1516_nums)

            # pers are on pages 3, 8, 13, ...
            if (i + 2) < pdf_doc.page_count:
                pers_page = pdf_doc.load_page(i + 2)
                pers_text = pers_page.get_text("text")
                clean_pers_data = clean_pers(pers_text)
                df_1516_pers = build_persdf(clean_pers_data)
                df_1516_pers_list.append(df_1516_pers)

        # Concatenate all individual DataFrames
        df_1516_tots = pd.concat(df_1516_tots_list, ignore_index=True)
        df_1516_nums = pd.concat(df_1516_nums_list, ignore_index=True)
        df_1516_pers = pd.concat(df_1516_pers_list, ignore_index=True)

        # Merge the DataFrames
        merged_df = merge(df_1516_nums, df_1516_pers, df_1516_tots)

        return merged_df

    finally:
        pdf_doc.close()

In [ ]:
pdf_path = "/content/MD_KRA_15-16.pdf"
md_df_1516 = get_tables(pdf_path)
md_df_1516

,name,subgroup,domain,n_demo,n_notdemo,n_appr,n_emer,pers_demo,pers_notdemo,pers_appr,pers_emer,n_tested,pers_tested
0,Maryland State,American Indian/Alaskan Native,lanlit,76,130,NaN,NaN,37,63,NaN,NaN,211,0.32
1,Maryland State,American Indian/Alaskan Native,math,83,123,NaN,NaN,40,60,NaN,NaN,211,0.32
2,Maryland State,American Indian/Alaskan Native,sel,108,98,NaN,NaN,52,48,NaN,NaN,211,0.32
3,Maryland State,American Indian/Alaskan Native,phy,113,94,NaN,NaN,55,45,NaN,NaN,211,0.32
4,Maryland State,American Indian/Alaskan Native,overall,87,NaN,77,36,44,NaN,39,18,211,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2987,Washington County,Predominant Prior Care - Final Record Count fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1583,
2988,Worcester County,Repeated Kindergarten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.21
2989,Worcester County,Final Record Count for KRA Data File,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,469,Kindergarten Readiness Assessment
2990,Maryland School for the Deaf,Repeated Kindergarten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00


In [ ]:
md_df_1516.to_csv("MD_1516.csv")

## 2016-17 (add in state)

In [ ]:
def get_constants(text):
  text_lines = text.split("\n")
  df = pd.DataFrame(columns = ["name", "tot_enrlmt", "p_rate", "subgroup",
                              "domain", "pers_demo", "pers_appr", "pers_emer"])
  name = tot_enrlmt = p_rate = ""
  for i in range(len(text_lines)):
    if text_lines[i] == '• KRA Assessed ':
      p_rate = text_lines[i + 1]
    if text_lines[i] == 'Total Students  ':
      tot_enrlmt = text_lines[i + 1]
    if text_lines[i] == "Maryland":
      name = text_lines[i + 1]
  return name, tot_enrlmt, p_rate

def get_data(text, name, tot_enrlmt, p_rate):
  text_lines = text.split("\n")
  df = pd.DataFrame(columns = ["name", "tot_enrlmt", "p_rate", "subgroup",
                              "domain", "pers_demo", "pers_appr", "pers_emer"])
  domains = ["lanlit", "math", "sel", "phy"]
  pers_demo_list = []
  row_index = 0
  subgroup = ""
  subgroups = ["children w/o disabilities", "children w/ disabilities",
              "english proficient", "english learners", "mid-high inc", "low inc"]
  for i in range(len(text_lines)):
    if text_lines[i] == "Maryland":
      name = text_lines[i + 1]
      #subgroup = "all"
      pers_emer = text_lines[i - 1]
      pers_appr = text_lines[i - 2]
      pers_demo_list.append(text_lines[i - 3])

      df.loc[row_index] = {
        "name": name,
        "tot_enrlmt": tot_enrlmt,
        "p_rate": p_rate,
        "subgroup": "all",
        "domain": "overall",
        "pers_demo": pers_demo_list[row_index],
        "pers_appr": pers_appr,
        "pers_emer": pers_emer
      }

      row_index += 1

    if text_lines[i] =='Language & ':
      domain_offsets = [4, 3, 2, 1]
      for j, domain in enumerate(domains):
        pers_demo_list.append(text_lines[i - domain_offsets[j]])
        df.loc[row_index] = {
            "name": name,
            "tot_enrlmt": tot_enrlmt,
            "p_rate": p_rate,
            "subgroup": "all",
            "domain": domain,
            "pers_demo": pers_demo_list[row_index],
            "pers_appr": "NA",
            "pers_emer": "NA"
        }
        row_index += 1

    if text_lines[i] == "Children w/o Disabilities":
      for n in range(1, 7):
        pers_demo_list.append(text_lines[i + n].lstrip(' '))
        df.loc[row_index] = {
            "name": name,
            "tot_enrlmt": tot_enrlmt,
            "p_rate": p_rate,
            "subgroup": subgroups[n - 1],
            "domain": "overall",
            "pers_demo": pers_demo_list[row_index],
            "pers_appr": "NA",
            "pers_emer": "NA"
        }
        row_index += 1
  return df

def process_pdf(pdf_path):
  document = fitz.open(pdf_path)
  combined_df = pd.DataFrame(columns=["name", "tot_enrlmt", "p_rate", "subgroup",
                                      "domain", "pers_demo", "pers_appr", "pers_emer"])

  for page_num in range(document.page_count):
      page = document.load_page(page_num)
      text = page.get_text()

      name, tot_enrlmt, p_rate = get_constants(text)
      df = get_data(text, name, tot_enrlmt, p_rate)

      combined_df = pd.concat([combined_df, df], ignore_index=True)

  return combined_df

In [ ]:
pdf_path = "/content/MD_1617_counties.pdf"
df_MD1617 = process_pdf(pdf_path)
df_MD1617

,name,tot_enrlmt,p_rate,subgroup,domain,pers_demo,pers_appr,pers_emer
0,Allegany County,629,100.0%,children w/o disabilities,overall,40,NA,NA
1,Allegany County,629,100.0%,children w/ disabilities,overall,14,NA,NA
2,Allegany County,629,100.0%,english proficient,overall,38,NA,NA
3,Allegany County,629,100.0%,english learners,overall,,NA,NA
4,Allegany County,629,100.0%,mid-high inc,overall,53,NA,NA
...,...,...,...,...,...,...,...,...
249,Worcester County,502,25.5%,english proficient,overall,47,NA,NA
250,Worcester County,502,25.5%,english learners,overall,*,NA,NA
251,Worcester County,502,25.5%,mid-high inc,overall,55,NA,NA
252,Worcester County,502,25.5%,low inc,overall,24,NA,NA


In [ ]:
df_MD1617.to_csv("MD_1617.csv")

## 2017-2018

In [ ]:
def get_constants(text):
  text_lines = text.split("\n")
  df = pd.DataFrame(columns = ["name", "tot_enrlmt", "p_rate", "subgroup",
                                "domain", "pers_demo", "pers_appr", "pers_emer"])
  name = tot_enrlmt = p_rate = ""
  for i in range(len(text_lines)):
    if text_lines[i] == "Emerging":
      name = text_lines[i + 1]
    if text_lines[i] == "READINESS":
      tot_enrlmt = text_lines[i + 1]
    if text_lines[i] == "KINDERGARTENERS ":
      p_rate = text_lines[i - 1]
  return name, tot_enrlmt, p_rate

In [ ]:
def get_data(text, name, tot_enrlmt, p_rate):
  text_lines = text.split("\n")
  df = pd.DataFrame(columns = ["name", "tot_enrlmt", "p_rate", "subgroup",
                                "domain", "pers_demo", "pers_appr", "pers_emer"])
  row_index = 0

  #domains = ["sel", "lanlit", "math", "phy", "overall"]
  domains = ["overall", "sel", "lanlit", "math", "phy"]
  pers_demo = []
  pers_appr = pers_emer = ""

  for i in range(len(text_lines)):
    if text_lines[i] == "DEMONSTRATE ":
      pers_demo.append(text_lines[i - 1])
    if text_lines[i] == "SOCIAL ":
      pers_demo.append(text_lines[i - 1])
    if text_lines[i] == "LANGUAGE & ":
      pers_demo.append(text_lines[i - 1])
    if text_lines[i] == "MATHEMATICS":
      pers_demo.append(text_lines[i - 1])
    if text_lines[i] == "PHYSICAL WELL-":
      pers_demo.append(text_lines[i - 1])
    if text_lines[i] == "Demonstrating":
      #pers_demo.append(text_lines[i - 6])
      pers_appr = text_lines[i - 4]
      pers_emer = text_lines[i - 5]


  for j, domain in enumerate(domains):
    df.loc[row_index] = {
        "name": name,
        "tot_enrlmt": tot_enrlmt,
        "p_rate": p_rate,
        "subgroup": "all",
        "domain": domains[j],
        "pers_demo": pers_demo[j] if j < len(pers_demo) else "NA",
        "pers_appr": pers_appr if domains[j] == "overall" else "NA",
        "pers_emer": pers_emer if domains[j] == "overall" else "NA"
    }
    row_index += 1

  races = ["blk", "asn", "hsp", "mtr", "wht"]

  subgroup_demo = []
  for i in range(len(text_lines)):
    if text_lines[i] == 'Race/Ethnicity DEMOGRAPHICS & READINESS':
      for n in range(2, 7):
        subgroup_demo.append(text_lines[i + n])

  for r, race in enumerate(races):
    df.loc[row_index] = {
        "name": name,
        "tot_enrlmt": tot_enrlmt,
        "p_rate": p_rate,
        "subgroup": races[r],
        "domain": "overall",
        "pers_demo": subgroup_demo[r] if r < len(subgroup_demo) else "NA",
        "pers_appr": "NA",
        "pers_emer": "NA"
    }
    row_index += 1

  # subtext = text_lines[text_lines.index('DISABILITIES'):text_lines.index('English Proficient Children') + 1]

  subgroups = ["iep", "nep", "ell", "nel", "frl", "nfl"]
  # subgroup_pers = ["iep", "nep", "ell", "nel", "frl", "nfl"]
  # initial_list = []
  # for i in range(len(subtext)):
  #   if subtext[i].isdigit() and subtext[i + 1] != 'Overall Readiness':
  #     initial_list.append(subtext[i])
  #     if 3 < len(initial_list) < 6:
  #       #Find whats missing
  #       if "Children w/Disabilities*" in subtext:
  #         #Add a * to the first 2 slots
  #         subgroup_pers[0] = "*"
  #         subgroup_pers[1] = "*"
  #         subgroup_pers[2] = initial_list[0]
  #         subgroup_pers[3] = initial_list[1]
  #         subgroup_pers[4] = initial_list[2]
  #         subgroup_pers[5] = initial_list[3]

  #       if subtext[i] == "English Learners*":
  #         #Add a * to the middle 2 slots
  #         subgroup_pers[0] = initial_list[0]
  #         subgroup_pers[1] = initial_list[1]
  #         subgroup_pers[2] = "*"
  #         subgroup_pers[3] = "*"
  #         subgroup_pers[4] = initial_list[2]
  #         subgroup_pers[5] = initial_list[3]

  #       #subgroup_pers.append(subtext[i])

  for s, subgroup in enumerate(subgroups):
    df.loc[row_index] = {
        "name": name,
        "tot_enrlmt": tot_enrlmt,
        "p_rate": p_rate,
        "subgroup": subgroups[s],
        "domain": "overall",
        "pers_demo": "fill",
        "pers_appr": "NA",
        "pers_emer": "NA"
    }
    row_index += 1

  return df

In [ ]:
def process_pdf(pdf_path):
  document = fitz.open(pdf_path)
  combined_df = pd.DataFrame(columns=["name", "tot_enrlmt", "p_rate", "subgroup",
                                      "domain", "pers_demo", "pers_appr", "pers_emer"])

  for page_num in range(22,47):
      page = document.load_page(page_num)
      text = page.get_text()

      name, tot_enrlmt, p_rate = get_constants(text)
      df = get_data(text, name, tot_enrlmt, p_rate)

      combined_df = pd.concat([combined_df, df], ignore_index=True)

  return combined_df

In [ ]:
pdf_path = "/content/readinessmatters_fullbook_2018.pdf"
df_1718 = process_pdf(pdf_path)
df_1718

,name,tot_enrlmt,p_rate,subgroup,domain,pers_demo,pers_appr,pers_emer
0,Allegany County,632,100%,all,overall,,41%,19%
1,Allegany County,632,100%,all,sel,57%,NA,NA
2,Allegany County,632,100%,all,lanlit,35%,NA,NA
3,Allegany County,632,100%,all,math,28%,NA,NA
4,Allegany County,632,100%,all,phy,48%,NA,NA
...,...,...,...,...,...,...,...,...
395,,,,nep,overall,fill,NA,NA
396,,,,ell,overall,fill,NA,NA
397,,,,nel,overall,fill,NA,NA
398,,,,frl,overall,fill,NA,NA


In [ ]:
df_1718.to_csv("raw_MD_2018.csv")

## 2018-2019

In [ ]:
def get_state_constants(text):
  text_lines = text.split("\n")
  # df = pd.DataFrame(columns = ["name", "tot_enrlmt", "p_rate", "subgroup",
  #                               "domain", "pers_demo", "pers_appr", "pers_emer", "score"])
  name = tot_enrlmt = p_rate = ""

  for i in range(len(text_lines)):
    if text_lines[i] == "STATEWIDE":
      name = text_lines[i - 1]
    if text_lines[i] == "ASSESSED BY KRA":
      p_rate = text_lines[i - 2]
      tot_enrlmt = text_lines[i + 9]

  return name, tot_enrlmt, p_rate

In [ ]:
def get_state_data(text, name, tot_enrlmt, p_rate):
  text_lines = text.split("\n")
  remove = ["298", "293"]
  text_lines = [line for line in text_lines if line not in remove]

  df = pd.DataFrame(columns = ["name", "tot_enrlmt", "p_rate", "subgroup",
                                "domain", "pers_demo", "pers_appr", "pers_emer", "score"])
  row_index = 0
  pers_demo = pers_appr = pers_emer = ""

  for i in range(len(text_lines)):
    if text_lines[i] == "Demonstrating":
      pers_demo = text_lines[i + 1]
      pers_appr = text_lines[i + 2]
      pers_emer = text_lines[i + 3]

      df.loc[row_index] = {
          "name": name,
          "tot_enrlmt": tot_enrlmt,
          "p_rate": p_rate,
          "subgroup": "all",
          "domain": "overall",
          "pers_demo": pers_demo,
          "pers_appr": pers_appr,
          "pers_emer": pers_emer,
          "score": "NA"
      }
      row_index += 1

  races = ["blk", "asn", "hsp", "mtr", "wht"]
  races_demo = []
  for i in range(len(text_lines)):
    if text_lines[i] == "Race/Ethnicity DEMOGRAPHICS & READINESS":
      for n in range(3, 8):
        races_demo.append(text_lines[i + n])
  for r, race in enumerate(races):
    df.loc[row_index] = {
        "name": name,
        "tot_enrlmt": tot_enrlmt,
        "p_rate": p_rate,
        "subgroup": races[r],
        "domain": "overall",
        "pers_demo": races_demo[r] if r < len(races_demo) else "NA",
        "pers_appr": "NA",
        "pers_emer": "NA",
        "score": "NA"
    }
    row_index += 1

  subgroups_demo = ["*","*","*","*","*","*"]
  subgroups = ["nep", "iep", "nel", "ell", "nfl", "frl"]
  for i in range(len(text_lines)):
    if text_lines[i] == "Children w/o ":
        subgroups_demo[0] = text_lines[i + 2]
        if "%" in text_lines[i + 3]:
          subgroups_demo[1] = text_lines[i + 3]
    if text_lines[i] == "w/Disabilities" and "%" in text_lines[i + 1]:
      subgroups_demo[1] = text_lines[i + 1]
      if text_lines[i] == "w/Disabilities*":
        subgroups_demo[1] = "*"

    if text_lines[i] == "Fluent":
      subgroups_demo[2] = text_lines[i + 2]
    if text_lines[i] == "English Learners":
      subgroups_demo[3] = text_lines[i + 1]
      if text_lines[i] == "English Learners*":
        subgroups_demo[3] = "*"

    if text_lines[i] == "Mid-/High-Income ":
      subgroups_demo[4] = text_lines[i + 2]
      if "%" in text_lines[i + 3]:
        subgroups_demo[5] = text_lines[i + 3]
    if text_lines[i] == "Low-Income":
      subgroups_demo[5] = text_lines[i + 2]

  for s, group in enumerate(subgroups):
    df.loc[row_index] = {
        "name": name,
        "tot_enrlmt": tot_enrlmt,
        "p_rate": p_rate,
        "subgroup": subgroups[s],
        "domain": "overall",
        "pers_demo": subgroups_demo[s] if s < len(subgroups_demo) else "NA",
        "pers_appr": "NA",
        "pers_emer": "NA",
        "score": "NA"
    }
    row_index += 1

  scores = ["x","x","x","x"]
  subscores = ["sel", "lanlit", "mth", "phy"]
  for i in range(len(text_lines)):
    if text_lines[i] == "Motor Development":
      scores[0] = text_lines[i + 3]
      scores[1] = text_lines[i + 1]
      scores[2] = text_lines[i + 4]
      scores[3] = text_lines[i + 2]
  for x, sub in enumerate(subscores):
    df.loc[row_index] = {
        "name": name,
        "tot_enrlmt": tot_enrlmt,
        "p_rate": p_rate,
        "subgroup": "all",
        "domain": subscores[x],
        "pers_demo": "NA",
        "pers_appr": "NA",
        "pers_emer": "NA",
        "score": scores[x] if x < len(scores) else "NA"
    }
    row_index += 1

  return df

In [ ]:
def get_constants(text):
  text_lines = text.split("\n")
  # df = pd.DataFrame(columns = ["name", "tot_enrlmt", "p_rate", "subgroup",
  #                               "domain", "pers_demo", "pers_appr", "pers_emer", "score"])
  name = tot_enrlmt = p_rate = ""

  for i in range(len(text_lines)):
    if text_lines[i] == 'READINESS ':
      name = text_lines[i + 1].strip().title()
      name = re.sub(r'\bCount Y\b', 'County', name, flags=re.IGNORECASE)
      name = re.sub(r'\bCit Y\b', 'City', name, flags=re.IGNORECASE)
    if text_lines[i] == 'ASSESSED':
      tot_enrlmt = text_lines[i + 1]
      p_rate = text_lines [i - 2]
  return name, tot_enrlmt, p_rate

In [ ]:
def get_data(text, name, tot_enrlmt, p_rate):
  text_lines = text.split("\n")
  remove = ["298", "293"]
  text_lines = [line for line in text_lines if line not in remove]

  df = pd.DataFrame(columns = ["name", "tot_enrlmt", "p_rate", "subgroup",
                                "domain", "pers_demo", "pers_appr", "pers_emer", "score"])
  row_index = 0
  pers_demo = pers_appr = pers_emer = ""

  for i in range(len(text_lines)):
    if text_lines[i] == "Demonstrating":
      pers_demo = text_lines[i + 1]
      if len(pers_demo.split()) > 1:
        pers_demo = pers_demo[0:3]
        pers_appr = text_lines[i + 2]
        pers_emer = text_lines[i + 5]
      else:
        pers_appr = text_lines[i + 3]
        pers_emer = text_lines[i + 6]
      if len(pers_appr.split()) > 1:
        pers_appr = pers_appr[0:3]
        pers_emer = text_lines[i + 4]
      if len(text_lines[i + 3].split()) > 1:
        pers_emer = text_lines[i + 5]

      df.loc[row_index] = {
            "name": name,
            "tot_enrlmt": tot_enrlmt,
            "p_rate": p_rate,
            "subgroup": "all",
            "domain": "overall",
            "pers_demo": pers_demo,
            "pers_appr": pers_appr,
            "pers_emer": pers_emer,
            "score": "NA"
      }
      row_index += 1

  races = ["blk", "asn", "hsp", "mtr", "wht"]
  races_demo = []
  for i in range(len(text_lines)):
    if text_lines[i] == "Race/Ethnicity DEMOGRAPHICS & READINESS":
      for n in range(3, 8):
        races_demo.append(text_lines[i + n])
  for r, race in enumerate(races):
    df.loc[row_index] = {
        "name": name,
        "tot_enrlmt": tot_enrlmt,
        "p_rate": p_rate,
        "subgroup": races[r],
        "domain": "overall",
        "pers_demo": races_demo[r] if r < len(races_demo) else "NA",
        "pers_appr": "NA",
        "pers_emer": "NA",
        "score": "NA"
    }
    row_index += 1

  subgroups_demo = ["*","*","*","*","*","*"]
  subgroups = ["nep", "iep", "nel", "ell", "nfl", "frl"]
  for i in range(len(text_lines)):
    if text_lines[i] == "Children w/o ":
        subgroups_demo[0] = text_lines[i + 2]
        if "%" in text_lines[i + 3]:
          subgroups_demo[1] = text_lines[i + 3]
    if text_lines[i] == "w/Disabilities" and "%" in text_lines[i + 1]:
      subgroups_demo[1] = text_lines[i + 1]
      if text_lines[i] == "w/Disabilities*":
        subgroups_demo[1] = "*"

    if text_lines[i] == "Fluent":
      subgroups_demo[2] = text_lines[i + 2]
    if text_lines[i] == "English Learners":
      subgroups_demo[3] = text_lines[i + 1]
      if text_lines[i] == "English Learners*":
        subgroups_demo[3] = "*"

    if text_lines[i] == "Mid-/High-Income ":
      subgroups_demo[4] = text_lines[i + 2]
      if "%" in text_lines[i + 3]:
        subgroups_demo[5] = text_lines[i + 3]
    if text_lines[i] == "Low-Income":
      subgroups_demo[5] = text_lines[i + 2]

  for s, group in enumerate(subgroups):
    df.loc[row_index] = {
        "name": name,
        "tot_enrlmt": tot_enrlmt,
        "p_rate": p_rate,
        "subgroup": subgroups[s],
        "domain": "overall",
        "pers_demo": subgroups_demo[s] if s < len(subgroups_demo) else "NA",
        "pers_appr": "NA",
        "pers_emer": "NA",
        "score": "NA"
    }
    row_index += 1

  scores = ["x","x","x","x"]
  subscores = ["sel", "lanlit", "mth", "phy"]
  for i in range(len(text_lines)):
    if text_lines[i] == "Motor Development":
    #sel
      if text_lines[i + 1] == "273":
        scores[0] = text_lines[i + 6] if text_lines[i + 6] == "273" else text_lines[i + 1]
        scores[0] = text_lines[i + 6] if text_lines[i + 5] == "273" else text_lines[i + 1]
      elif text_lines[i + 6] == "273":
          scores[0] = text_lines[i + 5] if text_lines[i + 5] == "273" else text_lines[i + 1]
      else:
          scores[0] = text_lines[i + 6]
      #lanlit & phy
      if text_lines[i + 2] == "268":
        scores[1] = text_lines[i + 2] if text_lines[i + 3] == "268" else text_lines[i + 1]
        scores[3] = text_lines[i + 4] if text_lines[i + 3] == "268" else text_lines[i + 3]
      else:
          scores[1] = text_lines[i + 2]
          scores[3] = text_lines[i + 4] if text_lines[i + 3] == "268" else None
      #mth
      if text_lines[i + 8] == "268":
        scores[2] = text_lines[i + 7]

  for x, sub in enumerate(subscores):
    df.loc[row_index] = {
        "name": name,
        "tot_enrlmt": tot_enrlmt,
        "p_rate": p_rate,
        "subgroup": "all",
        "domain": subscores[x],
        "pers_demo": "NA",
        "pers_appr": "NA",
        "pers_emer": "NA",
        "score": scores[x] if x < len(scores) else "NA"
    }
    row_index += 1


  return df

In [ ]:
def process_pdf(pdf_path):
  document = fitz.open(pdf_path)
  combined_df = pd.DataFrame(columns=["name", "tot_enrlmt", "p_rate", "subgroup",
                                      "domain", "pers_demo", "pers_appr", "pers_emer", "score"])
  pages = [3] + list(range(18, 43))

  for page_num in pages:
      page = document.load_page(page_num)
      text = page.get_text()

      if page_num == 3:
        name, tot_enrlmt, p_rate = get_state_constants(text)
        df = get_state_data(text, name, tot_enrlmt, p_rate)
      else:
        name, tot_enrlmt, p_rate = get_constants(text)
        df = get_data(text, name, tot_enrlmt, p_rate)
      combined_df = pd.concat([combined_df, df], ignore_index=True)

  return combined_df

In [ ]:
pdf_path = "/content/2018-19_rm_book.pdf"
df_1819 = process_pdf(pdf_path)
df_1819

,name,tot_enrlmt,p_rate,subgroup,domain,pers_demo,pers_appr,pers_emer,score
0,Maryland,"64,600",39%,all,overall,47%,33%,20%,NA
1,Maryland,"64,600",39%,blk,overall,42%,NA,NA,NA
2,Maryland,"64,600",39%,asn,overall,57%,NA,NA,NA
3,Maryland,"64,600",39%,hsp,overall,29%,NA,NA,NA
4,Maryland,"64,600",39%,mtr,overall,53%,NA,NA,NA
...,...,...,...,...,...,...,...,...,...
410,,,,frl,overall,*,NA,NA,NA
411,,,,all,sel,NA,NA,NA,x
412,,,,all,lanlit,NA,NA,NA,x
413,,,,all,mth,NA,NA,NA,x


In [ ]:
df_1819.to_csv("raw_MD_2019.csv")

## 2019-2020

In [ ]:
  def get_constants(text):

  name = tot_enrlmt = p_rate = ""

  text_lines = text.split("\n")
  remove = [" ", "0", "10", "20", "30", "40", "50", "60", "70", "80"]
  text_lines = [line for line in text_lines if line.strip() not in remove]

  for i in range(len(text_lines)):
    if "ADMINISTRATION METHOD:" in text_lines[i]:
      if "COUNTY" in text_lines[i + 2] or "CITY" in text_lines[i + 2]:
        name = (text_lines[i + 1] + text_lines[i + 2])
        tot_enrlmt = text_lines[i + 3]
        p_rate = text_lines[i + 4]
      else:
        name = text_lines[i + 1]
        tot_enrlmt = text_lines[i + 2]
        p_rate = text_lines[i + 3]
    if text_lines[i] == 'COUNTY ':
      tot_enrlmt = text_lines[i + 1]
      p_rate = text_lines[i + 2]
  return name, tot_enrlmt, p_rate

In [ ]:
def get_data(text, page_num, name, tot_enrlmt, p_rate):

  df = pd.DataFrame(columns=["name", "tot_enrlmt", "p_rate", "subgroup",
                                      "domain", "pers_demo", "pers_appr", "pers_emer", "score"])

  pers_demo = pers_appr = pers_emer = ""

  text_lines = text.split("\n")
  remove = [" ", "0", "10", "20", "30", "40", "50", "60", "70", "80"]
  text_lines = [line for line in text_lines if line.strip() not in remove]

  row_index = 0

  #demo, appr, emer
  for i in range(len(text_lines)):
    if page_num == 4:
      if text_lines[i] == "Emerging ":
        pers_demo = text_lines[i + 3]
        pers_appr = text_lines[i + 1]
        pers_emer = text_lines[i + 2]

    else:
      if "2018-2019" in text_lines[i]:
        #start = text_lines[i]
        if "2019-2020" in text_lines[i + 1]:
          pers_demo = text_lines[i + 2] if "47%" not in text_lines[i + 2] else text_lines[i + 3]
          if "32%" in text_lines[i + 4]:
            if len(text_lines[i + 4].split()) > 1:
              pers_appr = text_lines[i + 4]
              if "21%" in text_lines[i + 5]:
                pers_emer = text_lines[i + 6]
                if len(text_lines[i + 5].split()) > 1:
                  pers_emer = text_lines[i + 5]
              else:
                pers_emer = text_lines[i + 5]
            else:
              pers_appr = text_lines[i + 5]

              if "21%" in text_lines[i + 6]:
                if len(text_lines[i + 6].split()) > 1:
                  pers_emer = text_lines[i + 6]
                else:
                  pers_emer = text_lines[i + 7]
          else:
            pers_appr = text_lines[i + 4]
            if "21%" in text_lines[i + 6]:
              if len(text_lines[i + 6].split()) > 1:
                pers_emer = text_lines[i + 6]
              else:
                pers_emer = text_lines[i + 7]
        elif "2019-2020" in text_lines[i + 2]:
          pers_demo = text_lines[i + 1]
          pers_appr = text_lines[i + 4]
          pers_emer = text_lines[i + 6]

        if len(pers_appr.split()) > 1:
          pers_appr = pers_appr[0:3]
        if len(pers_emer.split()) > 1:
          pers_emer = pers_emer[0:3]

        break

  df.loc[row_index] = {
        "name": name,
        "tot_enrlmt": tot_enrlmt,
        "p_rate": p_rate,
        "subgroup": "all",
        "domain": "overall",
        "pers_demo": pers_demo,
        "pers_appr": pers_appr,
        "pers_emer": pers_emer,
        "score": "NA"
  }
  row_index += 1

  #races
  races = ["blk", "nam", "asn", "hsp", "hpi", "mtr", "wht"]
  races_demo = ["*","*","*","*","*","*", "*"]
  race_order = ["African American", "American Indian", "Asian",
                "Hispanic/Latino", "Native Hawaiian/Pacific Islander", "Two or More", "White"]
  for i in range(len(text_lines)):
    if page_num == 4:
      if text_lines[i].strip() == 'Kindergarten Readiness':
        for n in range(1, 8):
          races_demo[n - 1] = text_lines[i + n].strip()

    else:
      start_index = text_lines.index('Kindergarten Readiness ')
      end_index = text_lines.index('Kindergarten ', start_index)
      race_data = text_lines[start_index + 1:end_index]
      race_data = [x.strip() for x in race_data if x.strip()]

      races_start = end_index + 2
      race_labels = text_lines[races_start:]
      race_labels = [label.strip() for label in race_labels]

      r_index = 0
      for label in race_labels:
        label_clean = label.rstrip("*").strip()
        if label_clean in race_order:
          index = race_order.index(label_clean)
          if label.endswith("*"):
            races_demo[index] = "*"
          elif r_index < len(race_data):
            races_demo[index] = race_data[r_index]
            r_index += 1

  for r, race, in enumerate(races):
    df.loc[row_index] = {
          "name": name,
          "tot_enrlmt": tot_enrlmt,
          "p_rate": p_rate,
          "subgroup": races[r],
          "domain": "overall",
          "pers_demo": races_demo[r] if r < len(races_demo) else "NA",
          "pers_appr": "NA",
          "pers_emer": "NA",
          "score": "NA"
    }
    row_index += 1

  #subgroups
  subgroups_demo = ["*","*","*","*","*","*"]
  subgroups = ["nep", "iep", "nel", "ell", "nfl", "frl"]
  for i in range(len(text_lines)):
    if page_num == 4:
      if text_lines[i] == 'Demonstrating ':
        subgroups_demo[0] = text_lines[i + 2]
        subgroups_demo[1] = text_lines[i + 3]
        subgroups_demo[2] = text_lines[i + 6]
        subgroups_demo[3] = text_lines[i + 5]
        subgroups_demo[4] = text_lines[i + 8]
        subgroups_demo[5] = text_lines[i + 9]
    else:
      if text_lines[i].strip() == "Children Without":
        subgroups_demo[0] = text_lines[i + 2]
        if "%" in text_lines[i + 3].strip():
          subgroups_demo[1] = text_lines[i + 3]
        elif text_lines[i].strip() == "Children With":
          subgroups_demo[1] = text_lines[i + 2]
      if text_lines[i].strip() == "Children With":
        subgroups_demo[1] = text_lines[i + 2]
        if "%" in text_lines[i + 3].strip() and "-" not in text_lines[i + 3].strip():
          subgroups_demo[2] = text_lines[i + 3]
      if text_lines[i].strip() == "Fluent":
        if subgroups_demo[2] == "*":
          subgroups_demo[2] = text_lines[i + 2]
      if text_lines[i].strip() == "Learner":
        subgroups_demo[3] = text_lines[i + 2]
      if text_lines[i].strip() == "Who Are Not":
        subgroups_demo[4] = text_lines[i + 2]
      if text_lines[i].strip() == "Who Are":
        subgroups_demo[5] = text_lines[i + 2]

  for s, subgroup, in enumerate(subgroups):
    df.loc[row_index] = {
          "name": name,
          "tot_enrlmt": tot_enrlmt,
          "p_rate": p_rate,
          "subgroup": subgroups[s],
          "domain": "overall",
          "pers_demo": subgroups_demo[s] if s < len(subgroups_demo) else "NA",
          "pers_appr": "NA",
          "pers_emer": "NA",
          "score": "NA"
    }
    row_index += 1

  #scores
  scores = ["x","x","x","x"]
  subscores = ["sel", "lanlit", "mth", "phy"]
  for i in range(len(text_lines)):
    if text_lines[i].strip() == "Motor Development":
      scores[0] = text_lines[i + 1]
      scores[1] = text_lines[i + 3]
      scores[2] = text_lines[i + 5]
      scores[3] = text_lines[i + 7]

  for x, subscore, in enumerate(subscores):
    df.loc[row_index] = {
          "name": name,
          "tot_enrlmt": tot_enrlmt,
          "p_rate": p_rate,
          "subgroup": "all",
          "domain": subscores[x],
          "pers_demo": "NA",
          "pers_appr": "NA",
          "pers_emer": "NA",
          "score": scores[x] if x < len(scores) else "NA"
    }
    row_index += 1


  return df

In [ ]:
def process_pdf(pdf_path):
  document = fitz.open(pdf_path)
  combined_df = pd.DataFrame(columns=["name", "tot_enrlmt", "p_rate", "subgroup",
                                      "domain", "pers_demo", "pers_appr", "pers_emer", "score"])
  pages = [4] + list(range(17, 41))
  for page_num in pages:
    page = document.load_page(page_num)
    text = page.get_text()
    name, tot_enrlmt, p_rate = get_constants(text)
    df = get_data(text, page_num, name, tot_enrlmt, p_rate)
    combined_df = pd.concat([combined_df, df], ignore_index=True)
  return combined_df

In [ ]:
pdf_path = "/content/2019-20_ready5_book_web.pdf"
df_1920 = process_pdf(pdf_path)
df_1920

,name,tot_enrlmt,p_rate,subgroup,domain,pers_demo,pers_appr,pers_emer,score
0,MARYLAND,"65,012",65%,all,overall,47%,32%,21%,NA
1,MARYLAND,"65,012",65%,blk,overall,42%,NA,NA,NA
2,MARYLAND,"65,012",65%,nam,overall,41%,NA,NA,NA
3,MARYLAND,"65,012",65%,asn,overall,56%,NA,NA,NA
4,MARYLAND,"65,012",65%,hsp,overall,26%,NA,NA,NA
...,...,...,...,...,...,...,...,...,...
445,WORCESTER,460,100%,frl,overall,50%,NA,NA,NA
446,WORCESTER,460,100%,all,sel,NA,NA,NA,282
447,WORCESTER,460,100%,all,lanlit,NA,NA,NA,273
448,WORCESTER,460,100%,all,mth,NA,NA,NA,274


In [ ]:
df_1920.to_csv("raw_MD_2020.csv")

## 2021 - 2022

In [ ]:
def get_constants(text, page_num):

  name = tot_enrlmt = p_rate = ""

  text_lines = text.split("\n")

  for i in range(len(text_lines)):
    if "needs were identified in " in text_lines[i]:
      name = text_lines[i + 1].replace("?*", "")
    if 12 <= page_num <= 24:
      if "ADMINISTRATION METHOD:" in text_lines[i]:
        p_rate = text_lines[i].split()[2].strip()
      if text_lines[i] == "Overall READINESS":
        tot_enrlmt = text_lines[i + 1]
      else:
        if "ADMINISTRATION METHOD:" in text_lines[i]:
          tot_enrlmt = text_lines[i + 1] if "%" not in text_lines[i + 1] else text_lines[i + 4]
    else:
      if "ADMINISTRATION METHOD:" in text_lines[i]:
        tot_enrlmt = text_lines[i + 1] if "%" not in text_lines[i + 1] else text_lines[i + 4]
        p_rate = text_lines[i].split()[2].strip()

  return name, tot_enrlmt, p_rate

In [ ]:
def get_data(text, page_num, name, tot_enrlmt, p_rate):

  df = pd.DataFrame(columns=["name", "tot_enrlmt", "p_rate", "subgroup",
                                      "domain", "pers_demo", "pers_appr", "pers_emer", "score"])

  pers_demo = pers_appr = pers_emer = ""

  text_lines = text.split("\n")
  #remove = [" ", "0", "10", "20", "30", "40", "50", "60", "70", "80"]
  #text_lines = [line for line in text_lines if line.strip() not in remove]

  row_index = 0

  #demo
  for i in range(len(text_lines)):
    if "ADMINISTRATION METHOD:" in text_lines[i] and "%" in text_lines[i + 1]:
      pers_demo = text_lines[i + 1]
    if 12 <= page_num <= 24:
      if text_lines[i] == "Overall READINESS":
        pers_demo = text_lines[i + 3]
    else:
      if text_lines[i] == "Overall READINESS":
        pers_demo = text_lines[i + 1]



  df.loc[row_index] = {
      "name": name,
      "tot_enrlmt": tot_enrlmt,
      "p_rate": p_rate,
      "subgroup": "all",
      "domain": "overall",
      "pers_demo": pers_demo,
      "pers_appr": pers_appr,
      "pers_emer": pers_emer,
      "score": "NA"
  }
  row_index += 1

  #races
  races = ["nam", "asn", "blk", "hsp", "hpi", "mtr", "wht"]
  races_demo = []
  for i in range(len(text_lines)):
    if text_lines[i] == "Race/Ethnicity DEMOGRAPHICS & READINESS":
      for n in range(3, 10):
        races_demo.append(text_lines[i + n])

  for r, race in enumerate(races):
    df.loc[row_index] = {
        "name": name,
        "tot_enrlmt": tot_enrlmt,
        "p_rate": p_rate,
        "subgroup": races[r],
        "domain": "overall",
        "pers_demo": races_demo[r] if r < len(races_demo) else "NA",
        "pers_appr": "NA",
        "pers_emer": "NA",
        "score": "NA"
    }
    row_index += 1

  #subgroups
  if page_num == 11:
    subgroups = ["nfl", "frl", "nep", "iep", "nel", "ell"]
  else:
    subgroups = ["nfl", "frl", "iep", "ell", "nep", "nel"]

  subgroups_demo = []

  if page_num == 11:
    indices = [i for i, line in enumerate(text_lines) if "*Estimates" in line]
    if len(indices) >= 1:
      start = indices[0] + 1

    next_six = text_lines[start:start + 6]
    pers = [element for element in next_six if "%" in element]
    if len(pers) == 6:
      subgroups_demo.extend(pers)

  if page_num == 13:
    indices = [i for i, line in enumerate(text_lines) if line == "English Language Learners"]
    if len(indices) >= 1:
      start = indices[0] + 1

    next_six = text_lines[start:start + 6]
    pers = [element for element in next_six if "%" in element]
    if len(pers) == 6:
      subgroups_demo.extend(pers)

  if page_num in [0, 3, 7]:
    indices = [i for i, line in enumerate(text_lines) if line == "19-20"]
    if len(indices) >= 2:
      start = indices[1] + 1

    next_six = text_lines[start:start + 8]
    temp_list = ["x","x","x","x","x","x"]
    pers2 = [element for element in next_six if "%" in element]
    asts = [element for element in next_six if element == "*"]
    if len(asts) >= 2:
      temp_list[2] = "*"
      temp_list[3] = "*"
      temp_list[0] = pers2[0]
      temp_list[1] = pers2[1]
      temp_list[4] = pers2[2]
      temp_list[5] = pers2[3]
    if len(asts) == 1:
      temp_list[3] = "*"
      temp_list[0] = pers2[0]
      temp_list[1] = pers2[1]
      temp_list[2] = pers2[2]
      temp_list[4] = pers2[3]
      temp_list[5] = pers2[4]
    if "x" not in temp_list and len(temp_list) == 6:
      subgroups_demo.extend(temp_list)


  else:
    indices = [i for i, line in enumerate(text_lines) if line == "19-20"]
    if len(indices) >= 2:
      start = indices[1] + 1

    next_six = text_lines[start:start + 6]
    pers = [element for element in next_six if "%" in element]
    if len(pers) == 6:
      subgroups_demo.extend(pers)


    else:
      indices = [i for i, line in enumerate(text_lines) if "Not Direct Certiﬁed" in line]
      if len(indices) >= 1:
        start = indices[0]
      prev_six = text_lines[start - 6: start]
      pers = [element for element in prev_six if "%" in element or element == "*"]
      if len(pers) == 6:
        subgroups_demo.extend(pers)


  for s, subgroup, in enumerate(subgroups):
    df.loc[row_index] = {
          "name": name,
          "tot_enrlmt": tot_enrlmt,
          "p_rate": p_rate,
          "subgroup": subgroups[s],
          "domain": "overall",
          "pers_demo": subgroups_demo[s] if s < len(subgroups_demo) else "NA",
          "pers_appr": "NA",
          "pers_emer": "NA",
          "score": "NA"
    }
    row_index += 1


  #domain scores
  scores = ["x","x","x","x"]
  subscores = ["sel", "lanlit", "mth", "phy"]
  for i in range(len(text_lines)):
    if text_lines[i].strip() == "Motor Development":
      scores[0] = text_lines[i + 1]
      scores[1] = text_lines[i + 3]
      scores[2] = text_lines[i + 5]
      scores[3] = text_lines[i + 7]
  for x, subscore, in enumerate(subscores):
    df.loc[row_index] = {
          "name": name,
          "tot_enrlmt": tot_enrlmt,
          "p_rate": p_rate,
          "subgroup": "all",
          "domain": subscores[x],
          "pers_demo": "NA",
          "pers_appr": "NA",
          "pers_emer": "NA",
          "score": scores[x] if x < len(scores) else "NA"
    }
    row_index += 1


  return df

In [ ]:
def process_pdf(pdf_path):
  document = fitz.open(pdf_path)
  combined_df = pd.DataFrame(columns=["name", "tot_enrlmt", "p_rate", "subgroup",
                                      "domain", "pers_demo", "pers_appr", "pers_emer", "score"])
  pages = list(range(0, 24))
  for page_num in pages:
    page = document.load_page(page_num)
    text = page.get_text()
    name, tot_enrlmt, p_rate = get_constants(text, page_num)
    df = get_data(text, page_num, name, tot_enrlmt, p_rate)
    combined_df = pd.concat([combined_df, df], ignore_index=True)
  return combined_df
    #print(name, tot_enrlmt, p_rate)

## 2022 - 2023

In [ ]:
def get_constants(text, page_num):

  name = tot_enrlmt = ""
  p_rate = "NA"

  text_lines = text.split("\n")

  for i in range(len(text_lines)):
    if text_lines[i].strip() == "Maryland" and text_lines[i + 1].strip() == "21-22":
      name = text_lines[i + 2]
    if page_num in range(0, 12):
      if text_lines[i].strip() == "Overall READINESS":
        tot_enrlmt = text_lines[i + 1]
    else:
      if text_lines[i].strip() == "Subgroup READINESS":
        tot_enrlmt = text_lines[i + 1]

  return name, tot_enrlmt, p_rate

In [ ]:
def get_data(text, page_num, name, tot_enrlmt, p_rate):

  df = pd.DataFrame(columns=["name", "tot_enrlmt", "p_rate", "subgroup",
                                      "domain", "pers_demo", "pers_appr", "pers_emer", "score"])

  pers_demo = pers_appr = pers_emer = ""
  text_lines = text.split("\n")
  row_index = 0

  #demo, appr, emer
  for i in range(len(text_lines)):
    if text_lines[i].strip() == "Overall READINESS":
      if page_num in range(0, 12):
        pers_demo = text_lines[i + 3]
      else:
        pers_demo = text_lines[i + 1]

  for i in range(len(text_lines)):
    start = text_lines.index("22-23", text_lines.index("22-23") + 1)
    if page_num == 20:
      pers_emer = text_lines[start + 2]
      pers_appr = text_lines[start + 3]
    if page_num in range(1, 5) or page_num in range(14, 17):
      pers_emer = text_lines[start + 1]
      pers_appr = text_lines[start + 2]
    else:
      pers_emer = text_lines[start + 1]
      pers_appr = text_lines[start + 3]

  df.loc[row_index] = {
      "name": name,
      "tot_enrlmt": tot_enrlmt,
      "p_rate": p_rate,
      "subgroup": "all",
      "domain": "overall",
      "pers_demo": pers_demo,
      "pers_appr": pers_appr,
      "pers_emer": pers_emer,
      "score": "NA"
  }
  row_index += 1

  #domain scores
  scores = ["x","x","x","x"]
  subscores = ["sel", "lanlit", "mth", "phy"]
  for i in range(len(text_lines)):
    if text_lines[i].strip() == "Motor Development":
      scores[0] = text_lines[i + 1]
      scores[1] = text_lines[i + 3]
      scores[2] = text_lines[i + 5]
      scores[3] = text_lines[i + 7]
  for x, subscore, in enumerate(subscores):
    df.loc[row_index] = {
          "name": name,
          "tot_enrlmt": tot_enrlmt,
          "p_rate": p_rate,
          "subgroup": "all",
          "domain": subscores[x],
          "pers_demo": "NA",
          "pers_appr": "NA",
          "pers_emer": "NA",
          "score": scores[x] if x < len(scores) else "NA"
    }
    row_index += 1

  #races
  races = ["nam", "asn", "blk", "hsp", "hpi", "mtr", "wht"]
  races_demo = []
  for i in range(len(text_lines)):
    if text_lines[i] == "Race/Ethnicity DEMOGRAPHICS & READINESS":
      for n in range(3, 10):
        if text_lines[i + n].startswith("*") and len(text_lines[i + n]) > 1:
          races_demo.append("*")
        elif text_lines[i + n] == "*":
          races_demo.append(text_lines[i + n])
        else:
          races_demo.append(text_lines[i + n])

  for r, race in enumerate(races):
    df.loc[row_index] = {
        "name": name,
        "tot_enrlmt": tot_enrlmt,
        "p_rate": p_rate,
        "subgroup": races[r],
        "domain": "overall",
        "pers_demo": races_demo[r] if r < len(races_demo) else "NA",
        "pers_appr": "NA",
        "pers_emer": "NA",
        "score": "NA"
    }
    row_index += 1

  # subgroups

  subgroups = ["nfl", "frl", "iep", "ell", "nep", "nel"]
  subgroups_demo = []

  indices = [i for i, line in enumerate(text_lines) if "Not Direct Certiﬁed" in line]
  if len(indices) >= 1:
    start = indices[0]
  prev_six = text_lines[start - 6: start]
  pers = [element for element in prev_six if "%" in element or element == "*"]
  if pers.count("*") == 0:
    subgroups_demo.extend(pers)
  else:
    temp_list = ["x","x","x","x","x","x"]
    if page_num == 4 or page_num == 19:
      if pers.count("*") == 1: #[iep*, nfl, frl, ell, nep, nel]
        temp_list[2] = pers[0]
        temp_list[0] = pers[1]
        temp_list[1] = pers[2]
        temp_list[3] = pers[3]
        temp_list[4] = pers[4]
        temp_list[5] = pers[5]
      if pers.count("*") == 2: #[iep*, ell*, nfl, frl, nep, nel]
        temp_list[2] = pers[0]
        temp_list[3] = pers[1]
        temp_list[0] = pers[2]
        temp_list[1] = pers[3]
        temp_list[4] = pers[4]
        temp_list[5] = pers[5]
    if page_num in [5, 11, 12]: #[ell*, nfl, frl, iep, nep, nel]
      temp_list[3] = pers[0]
      temp_list[0] = pers[1]
      temp_list[1] = pers[2]
      temp_list[2] = pers[3]
      temp_list[4] = pers[4]
      temp_list[5] = pers[5]
    subgroups_demo.extend(temp_list)

  for s, subgroup, in enumerate(subgroups):
    df.loc[row_index] = {
          "name": name,
          "tot_enrlmt": tot_enrlmt,
          "p_rate": p_rate,
          "subgroup": subgroups[s],
          "domain": "overall",
          "pers_demo": subgroups_demo[s] if s < len(subgroups_demo) else "NA",
          "pers_appr": "NA",
          "pers_emer": "NA",
          "score": "NA"
    }
    row_index += 1


  return df

In [ ]:
def process_pdf(pdf_path):
  document = fitz.open(pdf_path)
  combined_df = pd.DataFrame(columns=["name", "tot_enrlmt", "p_rate", "subgroup",
                                      "domain", "pers_demo", "pers_appr", "pers_emer", "score"])
  pages = list(range(0, 24))
  for page_num in pages:
    page = document.load_page(page_num)
    text = page.get_text()
    name, tot_enrlmt, p_rate = get_constants(text, page_num)
    df = get_data(text, page_num, name, tot_enrlmt, p_rate)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

  return combined_df

In [ ]:
process_pdf(pdf_path)